---
title: "Instruction on generate_changelog.py script"
author: Vadim Rudakov, rudakow.wadim@gmail.com
date: 2026-02-18
options:
  version: 1.0.2
  birth: 2026-02-16
---

# Instruction on generate_changelog.py script

:::{important} Configuration source
This script reads its configuration — type-to-section mapping, section ordering — from `pyproject.toml [tool.commit-convention.changelog-sections]`. That section is the single source of truth shared with `validate_commit_msg.py`. The rules themselves are defined in [Production Git Workflow Standards](/tools/docs/git/01_production_git_workflow_standards.ipynb).
:::

## **1. Architectural Overview: The SVA Principle**

This [script](/tools/scripts/generate_changelog.py) generates hierarchical CHANGELOG entries from structured git commit bodies ({term}`ADR-26024`). It is the "cooking" step in the **Ingredients-First** pattern:

1. **At commit time**: `validate_commit_msg.py` ensures every commit body contains parseable bullets (the "ingredients").
2. **At release time**: `generate_changelog.py` extracts those bullets from git history and formats them into the project's CHANGELOG format.

The script produces deterministic, traceable output — every CHANGELOG line maps 1:1 to a commit body bullet. No LLM involved.

It adheres to the **Smallest Viable Architecture (SVA)** principle.

:::{hint} **SVA = right tool for the job**
SVA isn't about minimal *code* — it's about **minimal *cognitive and operational overhead***.

* **Zero External Dependencies**: Uses only the Python standard library (`argparse`, `subprocess`, `tomllib`, `pathlib`, `re`, `dataclasses`).
* **Configuration from pyproject.toml**: Type-to-section mapping and section ordering are loaded from `pyproject.toml [tool.commit-convention.changelog-sections]` — a single source of truth shared with `validate_commit_msg.py`.
* **`--first-parent` scanning**: Filters out feature branch noise, scanning only squashed trunk commits produced by Squash-and-Merge.
:::

## **2. Key Capabilities & Logic**

### CHANGELOG Output Format

The script produces the project's hierarchical CHANGELOG format:

```
release 2.5.0
* New Features:
    - Add login page
        - Created: auth/login.py — new login page
        - Updated: auth/urls.py — added login route
* Bug Fixes:
    - Correct token expiry
        - Fixed: auth/token.py — expiry was off by one
```

**Hierarchy:**

| Level | Format | Source |
|-------|--------|--------|
| Version header | `release X.Y.Z` or `Unreleased` | `--version` flag |
| Section | `* Section Name:` | `pyproject.toml [tool.commit-convention.changelog-sections]` |
| Topic | `    - Capitalized subject` | Commit subject line (4-space indent) |
| Sub-item | `        - Body bullet` | Commit body bullet (8-space indent) |

### Commit Parsing

Each commit is parsed from `git log --format=%H%n%s%n%b%nEND_COMMIT_MARKER`:

* **Hash** — first line
* **Subject** — second line, parsed for type, optional scope, and description
* **Body** — remaining lines, filtered for bullets only

**Excluded from bullets:**
* ArchTag lines (`ArchTag:TAG-NAME`)
* Git trailers (`Co-Authored-By: ...` after a blank line)
* Prose context lines

### Legacy Commit Handling

Commits predating the structured body convention (no bullets) are included with their subject line only — no sub-items. This ensures graceful degradation when scanning history that spans the convention adoption date.

### Section Ordering

Sections appear in the order defined by `pyproject.toml [tool.commit-convention.changelog-sections]` key order. Types not in the mapping are appended at the end with their type name capitalized.

## **3. Technical Architecture**

The script follows top-down design:

* **`main()`** — entry point, instantiates CLI
* **`generate_changelog(ref_range, version)`** — orchestrates parsing → grouping → formatting
* **`parse_commits(ref_range)`** — runs `git log --first-parent`, splits on marker, calls parser
* **`parse_single_commit(raw)`** — extracts hash, type, scope, subject, bullets from raw text
* **`group_by_type(commits)`** — groups `list[Commit]` into `dict[str, list[Commit]]`
* **`format_changelog(groups, version)`** — produces the hierarchical CHANGELOG string
* **`GenerateChangelogCLI`** — argument parsing, stdout/file output

The `Commit` dataclass holds: `hash`, `type`, `scope`, `subject`, `bullets`.

## **4. Operational Guide**

### Configuration Reference

* **Script**: `tools/scripts/generate_changelog.py`
* **Configuration**: `pyproject.toml [tool.commit-convention.changelog-sections]`
* **Standards**: [Production Git Workflow Standards](/tools/docs/git/01_production_git_workflow_standards.ipynb) § Release-Time CHANGELOG Generation

### Command Line Interface

```bash
generate_changelog.py REF_RANGE [--version VERSION] [--prepend FILE]
```

| Argument | Description | Default |
|----------|-------------|---------|
| `REF_RANGE` | Git ref range (e.g., `v2.4.0..HEAD`) | Required |
| `--version` | Version label for the header | `Unreleased` |
| `--prepend` | Prepend output to an existing file | stdout |

### Usage Examples

In [1]:
cd ../../..

#### Generate changelog from last tag to HEAD - dry run

In [2]:
env -u VIRTUAL_ENV uv run tools/scripts/generate_changelog.py v2.4.0..HEAD | head -n10

Unreleased
* New Features:
    - Add MHTML support to extract_html_text.py
        - Updated: `tools/scripts/extract_html_text.py` — added is_mhtml() detection, extract_html_from_mhtml() with quoted-printable decoding, SVG skip tag, auto-detection in main()
        - Updated: `tools/tests/test_extract_html_text.py` — added 22 new tests for MHTML parsing, SVG/base64 stripping, and integration (36 total, 85% coverage)
        - Updated: `tools/docs/scripts_instructions/extract_html_text_py_script.md` — documented MHTML support, SVG stripping, new test classes
        - Updated: `tools/docs/scripts_instructions/extract_html_text_py_script.ipynb` — synced via jupytext
    - Enforce ADR section whitelist with conditional and code-fence support
        - Updated: `architecture/adr/adr_config.yaml` — added `allowed_sections` (10 items) and `conditional_sections` (Rejection Rationale → rejected only)
        - Updated: `tools/scripts/check_adr.py` — added `ALLOWED_SECTIONS`, `CONDITIONAL_SECTI

#### With version label - dry run

In [3]:
env -u VIRTUAL_ENV uv run tools/scripts/generate_changelog.py v2.4.0..HEAD --version 2.5.0 | head -n5

release 2.5.0
* New Features:
    - Add MHTML support to extract_html_text.py
        - Updated: `tools/scripts/extract_html_text.py` — added is_mhtml() detection, extract_html_from_mhtml() with quoted-printable decoding, SVG skip tag, auto-detection in main()
        - Updated: `tools/tests/test_extract_html_text.py` — added 22 new tests for MHTML parsing, SVG/base64 stripping, and integration (36 total, 85% coverage)


#### Generate from concrete 3 commits - dry run

In [4]:
env -u VIRTUAL_ENV uv run tools/scripts/generate_changelog.py HEAD~10..HEAD~7 | head -n10

Unreleased
* Bug Fixes:
    - Install commit-msg hook in configure_repo.py setup
        - Updated: `tools/scripts/configure_repo.py` — added `run_precommit_install_commit_msg()` to install commit-msg stage hook via `--hook-type commit-msg`
        - Updated: `tools/tests/test_configure_repo.py` — added tests for commit-msg hook install success/failure and mid-chain failure paths
        - Updated: `tools/docs/git/02_pre_commit_hooks_and_staging_instruction_for_devel.md` — added commit-msg install command and warning about hook stage installation
        - Updated: `tools/docs/git/03_precommit_ci_validation_system.md` — added commit-msg install command and cross-reference to article 02; migrated frontmatter to ADR-26023
        - Updated: `tools/docs/scripts_instructions/configure_repo_py_script.md` — added commit-msg hook row to operations table; migrated frontmatter to ADR-26023
* Documentation:
    - Prefer .md over .ipynb for reading notebooks


#### Prepend to existing CHANGELOG

```bash
uv run tools/scripts/generate_changelog.py v2.4.0..HEAD --version 2.5.0 --prepend CHANGELOG
```

## **5. Test Suite**

The script is accompanied by a test suite (`test_generate_changelog.py`) with 56 tests covering:

* **Commit parsing**: Hash, type, scope, subject, bullet extraction, trailer/ArchTag exclusion
* **Grouping**: Type-based grouping, order preservation, empty input
* **Formatting**: Version header, section headers, indentation hierarchy, capitalization, section ordering, legacy commits
* **Git integration**: `--first-parent` flag, multi-commit parsing, empty ranges (mocked subprocess)
* **CLI integration**: Required arguments, stdout output, `--version` flag, `--prepend` file writing

### Running the Tests

```bash
# Run all tests
uv run pytest tools/tests/test_generate_changelog.py

# Run with coverage
uv run pytest tools/tests/test_generate_changelog.py --cov=tools.scripts.generate_changelog --cov-report=term-missing
```

In [1]:
cd ../../../

In [2]:
env -u VIRTUAL_ENV uv run pytest tools/tests/test_generate_changelog.py -q

........................................................                 [100%]
56 passed in 0.14s


In [3]:
env -u VIRTUAL_ENV uv run pytest tools/tests/test_generate_changelog.py --cov=tools.scripts.generate_changelog --cov-report=term-missing -q

........................................................                 [100%]
================================ tests coverage ================================
_______________ coverage: platform linux, python 3.13.5-final-0 ________________

Name                                  Stmts   Miss  Cover   Missing
-------------------------------------------------------------------
tools/scripts/generate_changelog.py     129     12    91%   110-111, 121-123, 144-145, 180, 193-195, 353
-------------------------------------------------------------------
TOTAL                                   129     12    91%
56 passed in 0.29s
